In [8]:
from pydrake.all import (
    MultibodyPlantConfig,
    Meshcat,
    StartMeshcat,
    DiagramBuilder,
    AddMultibodyPlant,
    Parser,
    LoadModelDirectives,
    ModelDirectives,
    ProcessModelDirectives,
    Diagram,
    Context,
    AddDefaultVisualization,
    RigidTransform,
    Quaternion,
    MultibodyPlant,
    JacobianWrtVariable,
    MathematicalProgram,
    eq,
    Solve,
    QueryObject,
    SpatialForce,
    MultibodyForces,
    SceneGraph,
    AutoDiffXd,
    SpatialForce_,
    MultibodyForces_,
    RigidBody
)
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# quickly setup a diagram
config = MultibodyPlantConfig()
builder = DiagramBuilder()
plant_f, scene_graph = AddMultibodyPlant(config, builder)
plant_f: MultibodyPlant = plant_f
scene_graph: SceneGraph = scene_graph
plant_f.mutable_gravity_field().set_gravity_vector([0, 0, 0]) # set gravity to zero

parser = Parser(plant_f, scene_graph)
parser.package_map().AddPackageXml('../package.xml')

directive_file = "../config/spinner.yaml"
directives: ModelDirectives = LoadModelDirectives(directive_file)
models = ProcessModelDirectives(directives, plant_f, parser)
plant_f.Finalize()

diagram: Diagram = builder.Build()
diagram_ad = diagram.ToAutoDiffXd()
plant_ad: MultibodyPlant = diagram_ad.GetSubsystemByName(plant_f.get_name())

diagram_context = diagram.CreateDefaultContext()
diagram_context_ad = diagram_ad.CreateDefaultContext()

context_f = plant_f.GetMyMutableContextFromRoot(diagram_context)
context_ad = plant_ad.GetMyMutableContextFromRoot(diagram_context_ad)

In [14]:
des_q = np.array([0,0,0])
q0 = np.array([1.0,1.0,0])
qdot0 = np.array([0,0,0])
v0 = np.array([0,0,0])

prog = MathematicalProgram()

horizon = 50
dt = 0.1
qs = prog.NewContinuousVariables(rows=3, cols=horizon, name="qs")
us = prog.NewContinuousVariables(rows=2, cols=horizon-1, name="us")
dHC = 0.1 
vd = 1/dHC  # reciprocal of hunt-crossley constant
mu = 0.5 # friction coefficient
vs = 0.05 # stiction toleranc3
sigma = 1.0
kspring = 1.0

# set initial conditions
prog.AddConstraint(eq(qs[:,0], q0))

# set vs and accels as lookups for qs
def calculate_vs(q):
    qs_var = q.reshape((3,horizon))
    if qs_var.dtype == float:
        plant = plant_f
        context = context_f
    else:
        plant = plant_ad
        context = context_ad
    vs = np.zeros((3,horizon),dtype=object)
    vs[:,0] = v0
    for i in range(1,horizon):
        plant.SetPositions(context, qs_var[:,i])
        vs[:,i] = plant.MapQDotToVelocity(context, qs_var[:,i] - qs_var[:,i-1])/dt # vdot = N_q * qdot
    return vs

def calculate_accels(vs_var):
    accels = np.zeros((3,horizon-1),dtype=object)
    for i in range(1,horizon-1):
        accels[:,i] = (vs_var[:,i+1] - vs_var[:,i])/dt
    return accels

def calculate_qdots(q):
    qs_var = q.reshape((3,horizon))
    qdots = np.zeros((3,horizon),dtype=object)
    qdots[:,0] = qdot0
    for i in range(1,horizon):
        qdots[:,i] = (qs_var[:,i] - qs_var[:,i-1])/dt
    return qdots

def custom_vel_cost(q):
    vs = calculate_vs(q)
    return sum(vs[0,:]**2) + sum(vs[1,:]**2)

prog.AddCost(sum(us[0,:]**2) + sum(us[1,:]**2)) # minimize control effort for 2R arms
prog.AddCost(100*(sum(qs[0,:]**2) + sum(qs[1,:]**2))) # minimize joint angles
prog.AddCost(custom_vel_cost, vars=qs.flatten()) # minimize cartesian finger velocities

def calculate_dissipation(vn, vd):
    term = vn/vd
    if term < 0:
        return 1 - term 
    elif 0 <= term < 2:
        return (vn/vd - 2)**2/4
    else:
        return 0

def calculate_vel_contact(q, dq, pair, inspector):
    if q.dtype == float:
        plant = plant_f
        context = context_f
    else:
        plant = plant_ad
        context = context_ad
    plant.SetPositions(context, q)
    plant.SetVelocities(context, dq)

    id_A = pair.id_A
    id_B = pair.id_B

    bodyA : RigidBody = plant.GetBodyFromFrameId(inspector.GetFrameId(id_A))
    bodyB : RigidBody = plant.GetBodyFromFrameId(inspector.GetFrameId(id_B))

    X_WA = plant.EvalBodyPoseInWorld(context, bodyA) # A to WORLD
    X_WB = plant.EvalBodyPoseInWorld(context, bodyB) # B to WORLD

    X_AGa: RigidTransform = inspector.GetPoseInFrame(id_A).cast[AutoDiffXd]() # Geometry_a to A
    X_BGb: RigidTransform = inspector.GetPoseInFrame(id_B).cast[AutoDiffXd]() # Geometry_b to B

    p_GaCa_Ga = pair.p_ACa # expressed in A frame
    X_WGa = X_WA @ X_AGa # Geometry_a to WORLD
    X_WCa_W = X_WGa @ p_GaCa_Ga # Contact point on A in WORLD

    p_GbCb_Gb = pair.p_BCb # expressed in B frame
    X_WGb = X_WB @ X_BGb # Geometry_b to WORLD
    X_WCb_W = X_WGb @ p_GbCb_Gb # Contact point on B in WORLD

    # contact point is midpoint between contact_A and contact_B
    p_WC = 0.5 * (X_WCa_W + X_WCb_W)

    # shift vector
    p_AC_W = p_WC - X_WA.translation()
    p_BC_W = p_WC - X_WB.translation()

    # velocities
    V_WA = plant.EvalBodySpatialVelocityInWorld(context, bodyA)
    V_WB = plant.EvalBodySpatialVelocityInWorld(context, bodyB)

    V_WAc = V_WA.Shift(p_AC_W)
    V_WBc = V_WB.Shift(p_BC_W)

    v_AcBc_W = V_WBc.translational() - V_WAc.translational()

    nhat = -pair.nhat_BA_W
    vn = nhat.dot(v_AcBc_W)
    vt = v_AcBc_W - vn * nhat
    
    return vn, vt, nhat, p_AC_W, p_BC_W, bodyA, bodyB

def calculate_tangential_component(mu, vt, vs, fn):
    return -mu * (vt/np.sqrt(vs ** 2 + np.linalg.norm(vt) ** 2)) * fn

def calculate_contact_force(q, dq, sigma, k, mu, vs,  thresh=5.0):
    if q.dtype == float:
        plant = plant_f
        context = context_f
    else:
        plant = plant_ad
        context = context_ad
    plant.SetPositions(context, q)
    plant.SetVelocities(context, dq)
    query_object : QueryObject = plant.get_geometry_query_input_port().Eval(context)
    sdf_pairs = query_object.ComputeSignedDistancePairwiseClosestPoints(thresh)
    pair = sdf_pairs[0]
    inspector = query_object.inspector()
    sdf = pair.distance

    c_sdf = sigma * k * np.log(1 + np.exp(-sdf/sigma))
    vn, vt, nhat, p_AC_W, p_BC_W, bodyA, bodyB = calculate_vel_contact(q, dq, pair, inspector)
    d = calculate_dissipation(vn, vd)
    fn = c_sdf * d
    ft = calculate_tangential_component(mu, vt, vs, fn)
    
    ft_BC_W = ft
    f_BC_W = nhat * fn + ft_BC_W
    
    # wrench/spatial force calculation
    F_BC_W = SpatialForce_[AutoDiffXd](np.zeros(3,dtype=object), f_BC_W)
    F_BBo_W = F_BC_W.Shift(-p_BC_W)
    
    F_AC_W = SpatialForce_[AutoDiffXd](np.zeros(3,dtype=object), -f_BC_W)
    F_AAo_W = F_AC_W.Shift(-p_AC_W)
    
    # now find spatial forces effect on Multibody equation (Manipulator Dynamics)
    mbf: MultibodyForces = MultibodyForces_[AutoDiffXd](plant.num_bodies(), plant.num_velocities())
    bodyB.AddInForceInWorld(context, F_BBo_W, mbf)
    bodyA.AddInForceInWorld(context, F_AAo_W, mbf)
    
    return mbf

# add inverse dynamics constraint
def custom_inverse_dynamics_eval(var):
    if var.dtype == float:
        plant = plant_f
        context = context_f
    else:
        plant = plant_ad
        context = context_ad
    q,u = np.split(var, [3*horizon])
    qs_var = q.reshape((3,horizon))
    us_var = u.reshape((2,horizon-1))
    vs_var = calculate_vs(q)
    accels = calculate_accels(vs_var)
    taus = np.zeros((2,horizon-1),dtype=object)
    for i in range(horizon-1):
        plant.SetPositions(context, qs_var[:,i+1])
        plant.SetVelocities(context, vs_var[:,i+1])
        
        # calculate contact forces
        mbf_ext = calculate_contact_force(qs_var[:,i+1], vs_var[:,i+1], sigma, kspring, mu, vs, thresh=5.0)
        
        tau = plant.CalcInverseDynamics(context, accels[:,i], mbf_ext)
        
        taus[:,i] = tau[:2]
    return (taus - us_var).flatten()
prog.AddConstraint(custom_inverse_dynamics_eval, lb=np.zeros(2*(horizon-1)), ub=np.zeros(2*(horizon-1)), vars=np.concatenate((qs.flatten(),us.flatten())))

In [15]:
result = Solve(prog)

print("Success? ", result.is_success())
print("Optimal cost: ", result.get_optimal_cost())
qs_sol = result.GetSolution(qs)

Success?  False
Optimal cost:  400.0


In [ ]:
# plot solution
plt.plot(qs_sol[0,:], label='q1')
plt.plot(qs_sol[1,:], label='q2')
plt.plot(qs_sol[2,:], label='q3')
plt.grid()
plt.legend()
plt.title("Joint Angles vs Time")
plt.show()